# Python Fortgeschritten: finally und Aufräumen
## Tag 2 - Notebook 11
***
In diesem Notebook wird behandelt:
- finally-Block
- with-Statement
- Context Manager
***


## 1 finally-Block

Der `finally`-Block wird immer ausgeführt, unabhängig davon, ob eine Exception auftritt oder nicht.

### Warum finally?

Der `finally`-Block garantiert, dass **Aufräumarbeiten** immer ausgeführt werden, auch wenn während der Ausführung ein Fehler auftritt. Das ist entscheidend für das **Ressourcen-Management** und verhindert, dass Ressourcen "hängen bleiben" oder in einem inkonsistenten Zustand verbleiben.

### Wann verwenden?

Der `finally`-Block sollte verwendet werden bei:
- **Ressourcen-Management**: Dateien müssen geschlossen werden, auch bei Fehlern
- **Netzwerkverbindungen**: Verbindungen müssen getrennt werden
- **Hardware-Zugriff**: Geräte müssen in einen sicheren Zustand zurückgesetzt werden
- **Status-Reset**: Globale Variablen oder Zustände müssen zurückgesetzt werden
- **Cleanup-Operationen**: Temporäre Dateien löschen, Caches leeren, etc.

### Vorteile

- **Verhindert Ressourcen-Leaks**: Dateien, Verbindungen und andere Ressourcen werden garantiert freigegeben
- **Konsistenter Zustand**: System bleibt in einem definierten Zustand, auch nach Fehlern
- **Zuverlässigkeit**: Cleanup-Code wird immer ausgeführt, unabhängig vom Ausführungspfad


In [ ]:
try:
    result = 10 / 2
    print(f"Ergebnis: {result}")
except ZeroDivisionError:
    print("Division durch Null!")
finally:
    print("Dies wird immer ausgeführt")


## 2 with-Statement

Das `with`-Statement stellt sicher, dass Ressourcen korrekt aufgeräumt werden.

### Warum with-Statement?

Das `with`-Statement bietet **automatisches Ressourcen-Management** und ist eleganter als manuelles `try/finally`. Es stellt sicher, dass Ressourcen automatisch geöffnet und geschlossen werden, ohne dass wir uns manuell darum kümmern müssen. Das ist die **Pythonic** Art, mit Ressourcen umzugehen.

### Wann verwenden?

Das `with`-Statement sollte verwendet werden bei allen Ressourcen, die geöffnet/geschlossen werden müssen:
- **Dateien**: `with open(...)` ist der Standard für Datei-Operationen
- **Datenbankverbindungen**: Automatisches Schließen von Verbindungen
- **Netzwerk-Sockets**: Automatisches Schließen von Verbindungen
- **Thread-Locks**: Automatisches Freigeben von Locks
- **Eigene Context Manager**: Für jede Ressource, die Initialisierung und Cleanup benötigt

### Vorteile

- **Weniger Code**: Kein manuelles `try/finally` nötig
- **Weniger Fehleranfällig**: Vergessenes Schließen von Ressourcen wird verhindert
- **Pythonic**: Entspricht Python-Best-Practices und ist idiomatisch
- **Lesbarkeit**: Code ist klarer und fokussiert sich auf die eigentliche Logik
- **Automatisch**: Ressourcen werden garantiert korrekt verwaltet, auch bei Exceptions


In [ ]:
# with-Statement für Dateien
with open("../data/test.txt", "w") as f:
    f.write("Test")

# Datei wird automatisch geschlossen, auch bei Fehlern


#### 5.1 Aufgaben:


### Aufgabe (a): Messgerät-Status-Management mit finally

Ein Messgerät durchläuft verschiedene Zustände: initialisiert → in Benutzung → aufgeräumt. Verwende `finally`, um sicherzustellen, dass der Status des Messgeräts immer korrekt zurückgesetzt wird.

**Anforderungen:**
- Die globale Variable `device_status` startet mit `"initialized"`
- In `perform_measurement()`: Setze `device_status = "in use"` (wird bereits gemacht)
- Die Funktion `perform_measurement()` kann einen `ValueError` werfen (bei `int("invalid")`)
- **Wichtig:** Im `finally`-Block **muss** `device_status` auf `"cleaned up"` gesetzt werden
- Der `finally`-Block wird **immer** ausgeführt, auch wenn eine Exception auftritt
- Nach dem try/except/finally-Block soll `device_status` immer `"cleaned up"` sein

**Erwartete Ausgabe:**
- Nach Ausführung: `device_status` sollte immer `"cleaned up"` sein, unabhängig davon, ob die Messung erfolgreich war oder fehlgeschlagen ist

**Hinweis:** `finally` wird immer ausgeführt, unabhängig von Exceptions


In [ ]:
device_status = "initialized"

def perform_measurement():
    """
    Simuliert eine Messung mit möglichen Fehlern.
    """
    global device_status
    device_status = "in use"
    
    # Simuliere Messung (kann ValueError werfen)
    measurement_value = int("invalid")  # Wird ValueError werfen
    return measurement_value

# TODO: Schritt 1 - Verwende try/except/finally
# TODO: Schritt 2 - Im try-Block: Rufe perform_measurement() auf und speichere Ergebnis
# TODO: Schritt 3 - Im try-Block: Gib "Messung erfolgreich: {result}" aus
# TODO: Schritt 4 - Im except-Block: Fange ValueError ab und gib "Fehler bei der Messung" aus
# TODO: Schritt 5 - Im finally-Block: Setze device_status = "cleaned up"
#       WICHTIG: Der finally-Block wird immer ausgeführt, auch wenn eine Exception auftritt!

try:
    result = perform_measurement()
    print(f"Messung erfolgreich: {result}")
except ValueError:
    print("Fehler bei der Messung")
finally:
    # TODO: Setze device_status hier auf "cleaned up"
    #       Dies wird IMMER ausgeführt, auch wenn eine Exception auftritt!
    pass

print(f"Gerätestatus: {device_status}")  # Erwartet: "cleaned up"


### Aufgabe (b): Messgerät-Context Manager

Ein Messgerät muss vor der Verwendung initialisiert und nach der Verwendung ordnungsgemäß geschlossen werden. Erstelle einen Context Manager, der dies automatisch übernimmt.

**Anforderungen:**
- Die Klasse `MeasurementDevice` ist bereits vorhanden, implementiere die Context Manager-Methoden:
- **`__enter__()` Methode:**
  - Setze `self.is_connected = True`
  - Gib `self` zurück (wichtig für `with ... as device:`)
- **`__exit__()` Methode:**
  - Parameter: `exc_type`, `exc_val`, `exc_tb` (Exception-Informationen)
  - Setze `self.is_connected = False`
  - Gib `False` zurück (bedeutet: Exception wird weitergeworfen, falls vorhanden)
- Verwende den Context Manager mit `with MeasurementDevice("Temperatursensor") as device:`
- Nach dem `with`-Block sollte `device.is_connected` automatisch `False` sein

**Erwartete Ausgabe:**
- Innerhalb des `with`-Blocks: `device.is_connected` sollte `True` sein
- Nach dem `with`-Block: `device.is_connected` sollte `False` sein (automatisch geschlossen)

**Hinweis:** Context Manager verwenden `__enter__()` und `__exit__()` Methoden


In [ ]:
class MeasurementDevice:
    """
    Context Manager für ein Messgerät.
    """
    def __init__(self, device_name):
        self.device_name = device_name
        self.is_connected = False
    
    # TODO: Schritt 1 - Implementiere __enter__() Methode
    #       def __enter__(self):
    #           # Setze self.is_connected = True
    #           # Gib self zurück (wichtig für 'with ... as device:')
    #           return self
    
    def __enter__(self):
        pass
    
    # TODO: Schritt 2 - Implementiere __exit__() Methode
    #       def __exit__(self, exc_type, exc_val, exc_tb):
    #           # exc_type, exc_val, exc_tb sind Exception-Informationen (können None sein)
    #           # Setze self.is_connected = False
    #           # Gib False zurück (bedeutet: Exception wird weitergeworfen, falls vorhanden)
    #           return False
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

# TODO: Schritt 3 - Verwende den Context Manager mit 'with'
#       with MeasurementDevice("Temperatursensor") as device:
#           print(f"Gerät {device.device_name} ist verbunden: {device.is_connected}")
#           print("Messung durchgeführt")
#       # Nach dem with-Block wird __exit__() automatisch aufgerufen
#       print(f"Gerät ist verbunden: {device.is_connected}")  # Sollte False sein

# Teste mit:
with MeasurementDevice("Temperatursensor") as device:
    print(f"Gerät {device.device_name} ist verbunden: {device.is_connected}")
    print("Messung durchgeführt")
# Nach dem with-Block sollte das Gerät automatisch geschlossen sein
print(f"Gerät ist verbunden: {device.is_connected}")  # Erwartet: False


### Aufgabe (c): Datenlogger mit finally und List Comprehension

Ein Datenlogger schreibt kontinuierlich Messwerte in eine Datei. Implementiere eine Funktion, die Messwerte verarbeitet, formatiert und in eine Datei schreibt.

**Anforderungen:**
- Die Funktion `log_measurements` soll Messwerte in eine Datei schreiben
- **Dateiformat:** Jede Zeile soll das Format haben: `"Messung {i+1}: {value:.2f}°C\n"`
  - `{i+1}`: Nummer der Messung (1-basiert)
  - `{value:.2f}`: Messwert mit 2 Dezimalstellen
  - Beispiel: `"Messung 1: 23.50°C\n"`
- **Formatierung:** Verwende List Comprehension: `[f"Messung {i+1}: {value:.2f}°C\n" for i, value in enumerate(measurements)]`
- **Datei-Handling:**
  - Öffne die Datei im `try`-Block: `file_handle = open(filename, "w")`
  - Schreibe formatierte Zeilen: `file_handle.writelines(formatted)`
  - Im `finally`-Block: Prüfe, ob `file_handle is not None`, dann schließe die Datei: `file_handle.close()`
- **Fehlerbehandlung:** Fange `IOError` ab und gib Fehlermeldung aus

**Erwartete Ausgabe:**
- Bei erfolgreichem Schreiben: `"{anzahl} Messungen in {filename} geschrieben"` und `"Datei geschlossen"`
- Bei Fehler: Fehlermeldung und trotzdem `"Datei geschlossen"` (dank finally)

**Hinweis:** List Comprehension kann verwendet werden, um Messwerte zu formatieren ([`List Comprehensions`](../1/01_comprehensions.ipynb))


In [ ]:
def log_measurements(measurements, filename="../data/measurements.log"):
    """
    Schreibt Messwerte in eine Log-Datei.
    
    Args:
        measurements: Liste von Messwerten
        filename: Pfad zur Log-Datei
    """
    file_handle = None
    
    try:
        # TODO: Schritt 1 - Öffne die Datei zum Schreiben: file_handle = open(filename, "w")
        # TODO: Schritt 2 - Formatiere Messwerte mit List Comprehension:
        #       formatted = [f"Messung {i+1}: {value:.2f}°C\n" for i, value in enumerate(measurements)]
        # TODO: Schritt 3 - Schreibe formatierte Zeilen: file_handle.writelines(formatted)
        # TODO: Schritt 4 - Gib aus: print(f"{len(measurements)} Messungen in {filename} geschrieben")
        pass
    
    except IOError as e:
        print(f"Fehler beim Schreiben der Datei: {e}")
    
    finally:
        # TODO: Schritt 5 - Prüfe, ob file_handle nicht None ist
        # TODO: Schritt 6 - Wenn nicht None: Schließe die Datei (file_handle.close())
        # TODO: Schritt 7 - Gib aus: print("Datei geschlossen")
        # WICHTIG: Der finally-Block wird immer ausgeführt, auch bei Fehlern!
        pass

# Teste mit:
test_measurements = [23.5, 24.1, 23.8, 24.3, 25.0]
log_measurements(test_measurements)
print("Messungen geloggt")  # Erwartet: "5 Messungen in ... geschrieben" und "Datei geschlossen"


### Aufgabe (d): Multi-Resource Context Manager mit itertools

Ein Messsystem verwendet mehrere Ressourcen gleichzeitig: eine Datei für Logging und eine Netzwerkverbindung für Datenübertragung. Beide müssen korrekt verwaltet werden.

**Anforderungen:**
- Die Klasse `MultiResourceManager` ist bereits vorhanden, implementiere die Context Manager-Methoden:
- **`__enter__()` Methode:**
  - Öffne die Log-Datei: `self.file_handle = open(self.log_file, "w")`
  - Setze `self.network_connected = True` (simuliert Netzwerkverbindung)
  - Gib `self` zurück
- **`__exit__()` Methode:**
  - Prüfe, ob `self.file_handle` existiert → schließe die Datei: `self.file_handle.close()`
  - Prüfe, ob `self.network_connected` ist → setze `self.network_connected = False`
  - Gib `False` zurück (Exception wird weitergeworfen)
- Die Funktion `process_measurement_data` soll:
  - `itertools.chain(*data_sources)` verwenden, um mehrere Listen zu kombinieren
  - In eine Liste konvertieren: `list(chain(*data_sources))`
  - Die kombinierte Liste zurückgeben

**Ressourcen-Management:**
- Beide Ressourcen (Datei und Netzwerk) werden automatisch im `__exit__()` geschlossen
- Auch bei Exceptions werden beide Ressourcen korrekt geschlossen

**Erwartete Ausgabe:**
- Nach dem `with`-Block: `manager.file_handle` sollte geschlossen sein und `manager.network_connected` sollte `False` sein

**Hinweis:** `itertools.chain` kann verwendet werden, um Daten aus verschiedenen Quellen zu kombinieren ([`itertools`](../1/03_itertools.ipynb))


In [ ]:
from itertools import chain

class MultiResourceManager:
    """
    Context Manager für mehrere Ressourcen (Datei + Netzwerkverbindung).
    """
    def __init__(self, log_file, network_address):
        self.log_file = log_file
        self.network_address = network_address
        self.file_handle = None
        self.network_connected = False
    
    # TODO: Schritt 1 - Implementiere __enter__()
    #       def __enter__(self):
    #           # Öffne die Log-Datei: self.file_handle = open(self.log_file, "w")
    #           # Setze self.network_connected = True
    #           # Gib self zurück
    #           return self
    
    def __enter__(self):
        pass
    
    # TODO: Schritt 2 - Implementiere __exit__()
    #       def __exit__(self, exc_type, exc_val, exc_tb):
    #           # Prüfe, ob self.file_handle existiert → schließe: self.file_handle.close()
    #           # Prüfe, ob self.network_connected → setze self.network_connected = False
    #           # Gib False zurück (Exception wird weitergeworfen)
    #           return False
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        pass
    
    def log_data(self, data):
        """Schreibt Daten in die Log-Datei."""
        if self.file_handle:
            self.file_handle.write(f"{data}\n")
    
    def send_data(self, data):
        """Simuliert das Senden von Daten über das Netzwerk."""
        if self.network_connected:
            print(f"Daten gesendet an {self.network_address}: {data}")

def process_measurement_data(*data_sources):
    """
    Verarbeitet Daten aus mehreren Quellen.
    
    Args:
        *data_sources: Variable Anzahl von Datenquellen (Listen)
    
    Returns:
        Kombinierte Daten (Liste)
    """
    # TODO: Schritt 3 - Verwende itertools.chain(*data_sources), um mehrere Listen zu kombinieren
    # TODO: Schritt 4 - Konvertiere zu Liste: list(chain(*data_sources))
    # TODO: Schritt 5 - Gib die kombinierte Liste zurück
    
    pass

# TODO: Schritt 6 - Verwende MultiResourceManager mit 'with'
#       sensor1_data = [23.5, 24.1, 23.8]
#       sensor2_data = [22.9, 23.2, 23.7]
#       
#       with MultiResourceManager("../data/system.log", "192.168.1.100") as manager:
#           combined_data = process_measurement_data(sensor1_data, sensor2_data)
#           for data_point in combined_data:
#               manager.log_data(f"Messwert: {data_point}")
#               manager.send_data(data_point)
#       # Nach dem with-Block werden beide Ressourcen automatisch geschlossen

# Teste mit:
sensor1_data = [23.5, 24.1, 23.8]
sensor2_data = [22.9, 23.2, 23.7]

with MultiResourceManager("../data/system.log", "192.168.1.100") as manager:
    # Kombiniere Daten aus verschiedenen Quellen
    combined_data = process_measurement_data(sensor1_data, sensor2_data)
    
    # Logge und sende Daten
    for data_point in combined_data:
        manager.log_data(f"Messwert: {data_point}")
        manager.send_data(data_point)
# Nach dem with-Block sollten beide Ressourcen automatisch geschlossen sein


#### Lösung:


In [ ]:
# Musterlösung (a): Messgerät-Status-Management
device_status = "initialized"

def perform_measurement():
    """
    Simuliert eine Messung mit möglichen Fehlern.
    """
    global device_status
    device_status = "in use"
    
    # Simuliere Messung (kann ValueError werfen)
    measurement_value = int("invalid")  # Wird ValueError werfen
    return measurement_value

try:
    result = perform_measurement()
    print(f"Messung erfolgreich: {result}")
except ValueError:
    print("Fehler bei der Messung")
finally:
    # finally wird immer ausgeführt, auch bei Exceptions
    device_status = "cleaned up"
    print(f"Gerät aufgeräumt")

print(f"Gerätestatus: {device_status}")  # Immer "cleaned up"


# Musterlösung (b): Messgerät-Context Manager
class MeasurementDevice:
    """
    Context Manager für ein Messgerät.
    """
    def __init__(self, device_name):
        self.device_name = device_name
        self.is_connected = False
    
    def __enter__(self):
        """Initialisiert das Gerät beim Eintritt in den Context."""
        print(f"Initialisiere {self.device_name}...")
        self.is_connected = True
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Schließt das Gerät beim Verlassen des Contexts."""
        print(f"Schließe {self.device_name}...")
        self.is_connected = False
        # False bedeutet: Exception wird weitergeworfen (falls vorhanden)
        return False

# Teste mit:
with MeasurementDevice("Temperatursensor") as device:
    print(f"Gerät {device.device_name} ist verbunden: {device.is_connected}")
    print("Messung durchgeführt")
# Nach dem with-Block ist das Gerät automatisch geschlossen
print(f"Gerät ist verbunden: {device.is_connected}")  # False


# Musterlösung (c): Datenlogger mit finally
def log_measurements(measurements, filename="../data/measurements.log"):
    """
    Schreibt Messwerte in eine Log-Datei.
    """
    file_handle = None
    
    try:
        # Öffne Datei
        file_handle = open(filename, "w")
        
        # Formatiere Messwerte mit List Comprehension
        formatted = [
            f"Messung {i+1}: {value:.2f}°C\n" 
            for i, value in enumerate(measurements)
        ]
        
        # Schreibe in Datei
        file_handle.writelines(formatted)
        print(f"{len(measurements)} Messungen in {filename} geschrieben")
    
    except IOError as e:
        print(f"Fehler beim Schreiben der Datei: {e}")
    
    finally:
        # Datei wird immer geschlossen, auch bei Fehlern
        if file_handle is not None:
            file_handle.close()
            print("Datei geschlossen")

# Teste mit:
test_measurements = [23.5, 24.1, 23.8, 24.3, 25.0]
log_measurements(test_measurements)


# Musterlösung (d): Multi-Resource Context Manager
from itertools import chain

class MultiResourceManager:
    """
    Context Manager für mehrere Ressourcen (Datei + Netzwerkverbindung).
    """
    def __init__(self, log_file, network_address):
        self.log_file = log_file
        self.network_address = network_address
        self.file_handle = None
        self.network_connected = False
    
    def __enter__(self):
        """Öffnet alle Ressourcen."""
        print(f"Öffne Log-Datei: {self.log_file}")
        self.file_handle = open(self.log_file, "w")
        
        print(f"Verbinde mit Netzwerk: {self.network_address}")
        self.network_connected = True
        
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Schließt alle Ressourcen."""
        if self.file_handle:
            print("Schließe Log-Datei")
            self.file_handle.close()
        
        if self.network_connected:
            print("Trenne Netzwerkverbindung")
            self.network_connected = False
        
        return False  # Exception weiterwerfen
    
    def log_data(self, data):
        """Schreibt Daten in die Log-Datei."""
        if self.file_handle:
            self.file_handle.write(f"{data}\n")
    
    def send_data(self, data):
        """Simuliert das Senden von Daten über das Netzwerk."""
        if self.network_connected:
            print(f"Daten gesendet an {self.network_address}: {data}")

def process_measurement_data(*data_sources):
    """
    Verarbeitet Daten aus mehreren Quellen mit itertools.chain.
    """
    # itertools.chain kombiniert mehrere Iterables zu einem
    combined = list(chain(*data_sources))
    return combined

# Teste mit:
sensor1_data = [23.5, 24.1, 23.8]
sensor2_data = [22.9, 23.2, 23.7]

with MultiResourceManager("../data/system.log", "192.168.1.100") as manager:
    # Kombiniere Daten aus verschiedenen Quellen
    combined_data = process_measurement_data(sensor1_data, sensor2_data)
    
    # Logge und sende Daten
    for data_point in combined_data:
        manager.log_data(f"Messwert: {data_point}")
        manager.send_data(data_point)

# Nach dem with-Block sind beide Ressourcen automatisch geschlossen
